In [0]:
### Ingestion File Movie Language

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment =  dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date =  dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

movie_language_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("languageId", IntegerType(), True),
    StructField("languageRoleId", IntegerType(), True)
])

movie_language = spark\
    .read\
    .schema(movie_language_schema)\
    .option("multiLine", True)\
    .json(f"{bronze_folder_path}/{v_file_date}/movie_language")

movie_language_df = movie_language\
    .withColumnRenamed("movieId", "movie_id")\
    .withColumnRenamed("languageId", "language_id")\
    .withColumn("environment", lit(v_environment))\
    .withColumn("file_date", lit(v_file_date))\
    .drop("languageRoleId")

movie_language_df = add_ingestion_date(movie_language_df)
# overwrite_partition(movie_language_df, "movie_silver", "movie_language", "file_date")

merge_condition = "tgt.movie_id = src.movie_id AND tgt.language_id = src.language_id AND tgt.file_date = src.file_date"
merge_delta_lake(movie_language_df, "movie_silver", "movie_language", silver_folder_path, merge_condition, "file_date")

# movie_language_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movie_language")


In [0]:
%sql
select file_date, count(1) 
from movie_silver.movie_language
group by file_date;

In [0]:
%sql
select *
from movie_silver.movie_language;

In [0]:
dbutils.notebook.exit("Success")